<a href="https://colab.research.google.com/github/pereirarodrigo/q_learning/blob/main/q_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement learning: Q-learning applied to a self-driving taxi cab
-------------
> This is a notebook dedicated to Satwik and Brendan's article on Reinforcement Learning, named "Reinforcement Q Learning from scratch in Python with OpenAI Gym" (https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/).

> The idea of this tutorial is to apply Q learning to a self-driving taxi cab, whose objective is to pick up 4 passengers in a 5x5 grid.

In [ ]:
# Installing OpenAI Gym, a library of game environments

!pip install cmake 'gym[atari]' scipy

In [ ]:
# Loading the game environment and rendering it
# Note: Taxi-v2 has been superseded by Taxi-v3

import gym

env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## The problem
--------
The taxi problem is written as follows (from the official Gym docs):

<br>

> "There are 4 locations (labeled by different letters), and our job is to pick up the passenger at one location and drop him off at another. We receive +20 points for a successful drop-off and lose 1 point for every time-step it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions."

<br>

Additionally, we have the following information:

<br>

*   The filled square represents the taxi. If it has no passenger, it is yellow; otherwise, it's green;
*   The pipe (|) represents a wall, which the taxi cannot pass;
*   R, G, Y, B are the possible pickup and destination locations. The blue letter represents the current passenger pick-up location, and the purple letter is the current destination.

In [ ]:
env.reset() # resetting the environment to a new, random state
env.render()

print(f"Action state: {env.action_space}")   # 6 action spaces (0 = south, 1 = north, 2 = east, 3 = west, 4 = pickup, 5 = dropoff)

print(f"State space: {env.observation_space}") # 500 state spaces (5 passengers * 5x5 taxi locations * 4 destinations)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action state: Discrete(6)
State space: Discrete(500)


In [ ]:
# Using the illustration's coordinates to set the environment's state, which is between 0 and 499
# In this case, the state is 209
# It is possible to set the environment's state manually with env.env.s using that number  

state = env.encode(2, 0, 2, 1)  # Taxi is at row 2 and column 0, passenger is at 2 and the destination is location 1 (R = 0, G = 1, Y = 2, B = 3)

print("State: ", state)

env.s = state  # setting the state 
env.render()

State:  209
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [ ]:
# Showing the reward table for state 209
# This table is created when the taxi environment is created
# It can be thought of as a states x actions matrix
# Its structure is: {action: [(probability, nextstate, reward, done)]}
# In our environment, the probability will always be 1 (100%)
# The nextstate is the state we would be in if we take the action at this index of the dictionary
# All movements have a reward of -1, while pickup/dropoff have a reward of -10
# If the taxi has a passenger and is located in a state with the right dropoff location, the total reward is 20
# Done tells us when the taxi has successfully dropped off a passenger in the right location. Each successful dropoff is the end of the episode

env.P[209]

{0: [(1.0, 309, -1, False)],
 1: [(1.0, 109, -1, False)],
 2: [(1.0, 229, -1, False)],
 3: [(1.0, 209, -1, False)],
 4: [(1.0, 209, -10, False)],
 5: [(1.0, 209, -10, False)]}

## Solving the problem without Reinforcement Learning
----------
It's possible to brute-force our way through this problem and solve it without using Reinforcement Learning. Given we have our $P$ table of default rewards in each state, we can make our taxi navigate by using only that. However, we will soon see that this approach is abysmally inefficient - the agent will make a lot of wrong dropoffs and take too long to do its task.

The way we do this is through an infinite loop which runs until one passenger reaches their destination, thus completing one episode, which is when the received reward is 20. The ```env.action_space.sample()``` method automatically selects one random action from the set of all possible actions. With this in mind, let's see what happens:

In [ ]:
env.s = 209  # manually setting the environment
epochs = 0
penalties, reward = 0, 0

frames = [] # for the animation
done = False

while not done:
  action = env.action_space.sample()
  state, reward, done, info = env.step(action)

  if reward == -10:
    penalties += 1

  # Putting each rendered frame into a dict for the animation

  frames.append({
      'frame' : env.render(mode = 'ansi'),
      'state' : state,
      'action' : action,
      'reward' : reward
      }
    )

  epochs += 1

print(f"Timesteps taken: {epochs}") 
print(f"Penalties incurred: {penalties}")

Timesteps taken: 315
Penalties incurred: 92


In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames, speed = .1):
  for i, frame in enumerate(frames):
    clear_output(wait = True)
    
    print(frame.get("frame"))
    print(f"Timestep: {i + 1}")
    print(f"State: {frame['state']}")
    print(f"Action: {frame['action']}")
    print(f"Reward: {frame['reward']}")

    sleep(speed)

print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 315
State: 85
Action: 5
Reward: 20


## Using Q-learning for a better solution
--------
Q-learning is a simple Reinforcement Learning algorithm that allows an agent to use its environment's rewards in order to learn, over time, the best action to take in a given state.

In our taxi environment, $P$, the reward table, is the means by which the agent will learn about solving its task. It does this by receiving a reward from for taking an action in the current state, then updating a Q-value to remember if that action was beneficial. Q-values are valures store in a Q-table, and they map to a ```(state, action)``` combination.

A Q-value for a particular state-action combination represents the "quality" of an action taken in that state. Thus, better Q-values imply better chances of getting better rewards. For example, if the taxi is faced with a state that includes a passenger at its current location, the Q-value for ```pickup``` will be significantly higher when compared to other actions, such as ```dropoff``` or moving to another direction.

Q-values are initialized to an arbitrary value (such as 0) and, as the agent exposes itself to the environment and receives different rewards by executing different actions, the Q-values are updated using the equation:

<br>

$$Q(state, action) \leftarrow (1 - \alpha)Q(state, action) + \alpha(reward + \gamma \: \underset{\alpha}{\mathrm{max}}\:Q(\text{next state, all actions}))$$

<br>

Where:

<br>

*    $\alpha$ = learning rate ($0 < \alpha \leq 1$), similar to supervised learning. It's the extent to which the Q-values are being updated;

*   $\gamma$ = discount factor ($0 \leq \gamma \leq 1$), which determines how much importance we want to give future rewards. A high value for the discount factor (close to 1) captures the long-term effective award, whereas a smaller factor value (close to 0) makes our agent consider only the immediate reward, thus making it greedy.

<br>

We are assigning ($\leftarrow$), or updating, the Q-value of the agent's current state and action by first taking a weight ($1 - \alpha$) of the old Q-value, then adding the learned value, which is a combination of the reward for taking the current action in the current state. The discounted maximum reward from the next state will be in once we take the current action.

After obtaining our Q-values, we need to store them in a Q-table, which has a row for every state (in our case, 500) and a column for every action (6). Its values are initially 0 and get updated after training. Our Q-table looks like this:

[Inserting an image]: ![](https://storage.googleapis.com/lds-media/images/q-matrix-initialized-to-learned_gQq0BFs.width-1200.png)

<center><img src = "https://storage.googleapis.com/lds-media/images/q-matrix-initialized-to-learned_gQq0BFs.width-1200.png" width = "600" height = "600">

<br>

<font size="3"> Source: https://storage.googleapis.com/lds-media/images/q-matrix-initialized-to-learned_gQq0BFs.width-1200.png</font></center>

<br>

Overall, the steps required for Q-learning are:

<br>

1.   Initialize the Q-table with 0;
2.   Start exploring actions: for each state, select any one among all possible actions for the current state $S$;
3.   Travel to the next state $S'$ as a result of action $a$;
4.   For all possible actions from the state $S'$, select the one with the highest Q-value;
5.   Update Q-table values using the equation;
6.   Set the next state as the current state;
7.   If the goal state is reached, then end and repeat the process. 

<br>

With this information at hand, it's time to implement Q-learning in Python.

In [ ]:
# Creating a 500x6 matrix of zeroes

import numpy as np

q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [ ]:
%%time

# Training the algorithm

import random
from IPython.display import clear_output

# Hyperparameters

alpha = 0.1
gamma = 0.6
epsilon = 0.1  # balancing exploration vs. exploitation, or choosing randomly vs. following known information
               # the smaller epsilon is, the more the agent will explore (thus resulting in more penalties)

# For plotting metrics

all_epochs = []
all_penalties = []

for i in range(1, 100001):
  state = env.reset()

  epochs, penalties, reward, = 0, 0, 0
  done = False

  while not done:
    if random.uniform(0, 1) < epsilon:
      action = env.action_space.sample()  # explore the action space

    else:
      action = np.argmax(q_table[state])  # exploit the learned value

    next_state, reward, done, info = env.step(action)

    old_value = q_table[state, action]
    next_max = np.max(q_table[next_state])
        
    new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
    q_table[state, action] = new_value

    if reward == -10:
      penalties += 1

    state = next_state
    epochs += 1
        
    if i % 100 == 0:
      clear_output(wait = True)
      print(f"Episode: {i}")

print("Training finished.\n")


Episode: 100000
Training finished.

CPU times: user 1min 38s, sys: 7.16 s, total: 1min 45s
Wall time: 2min 20s


In [ ]:
# Checking the Q-values at our illustration's state
# The max Q-value is "south" (-2.418), which means that the agent has succeeded in learning

q_table[209]

array([ -2.41837066,  -2.47061343,  -2.47061344,  -2.45102239,
       -11.45101854, -11.45102188])

In [ ]:
# Evaluating the agent's performance after training

total_epochs, total_penalties = 0, 0
episodes = 1  # change to however many episodes we'd like to test
frames = []

for _ in range(episodes):
  state = env.reset()
  epochs, penalties, reward = 0, 0, 0
  done = False

  while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)

    if reward == -10:
      penalties += 1

    # Putting each rendered frame into a dict for the animation

    frames.append({
      'frame' : env.render(mode = 'ansi'),
      'state' : state,
      'action' : action,
      'reward' : reward
      }
    )

    epochs += 1

  total_penalties += penalties
  total_epochs = epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 1 episodes:
Average timesteps per episode: 7.0
Average penalties per episode: 0.0


In [ ]:
print_frames(frames, 0.6)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 7
State: 410
Action: 5
Reward: 20
